# Construcción de una Base de Datos Vectorial desde cero

Una empresa llamada DataFind quiere crear su propia base de datos vectorial interna, sin depender de servicios externos.
Necesitan almacenar, indexar y buscar información basada en significado, no solo por coincidencia de texto.

Tu misión es simular una BDV completa capaz de:

Almacenar embeddings.

Buscar por similitud.

Implementar filtros por metadatos.

Medir la eficiencia y calidad de los resultados.

## Objetivos de la práctica

1. Generar embeddings simulados

- Cargar csv documentos_datafind.csv.

- Convertirlos en vectores de alta dimensión (usando numpy para simular embeddings).

2. Implementar componentes internos de la BDV:

- Storage Engine: guardar los vectores y metadatos.

- Index Builder: crear índices con estrategias de agrupamiento (simular IVF, HNSW o PQ).

- Query Engine: calcular similitudes (cosine y euclidean).

- Metadata Store: almacenar texto original, etiquetas o categorías.

3. Implementar consultas vectoriales

- Permitir que el usuario escriba una consulta y el sistema devuelva los documentos más similares.

- Simular embeddings de la consulta con el mismo método.

4. Agregar filtrado por metadatos

- Permitir buscar solo en documentos de una categoría (ej. “Tecnología”, “Salud”, “Educación”).

5. Evaluar el rendimiento y calidad de la búsqueda:

- Calcular tiempo de consulta.

- Evaluar qué métrica (cosine o euclidean) devuelve resultados más relevantes.

6. Simular optimización del índice:

- Comparar tiempos de búsqueda con distintas estrategias (sin índice, con IVF, con HNSW).

- Analizar cuál ofrece el mejor equilibrio entre velocidad y precisión.

7. Generar un informe final automático:

- Guardar resultados de las consultas, métricas y tiempos en un archivo reporte_resultados.csv.

In [1]:
# Cargar csv documentos_datafind.csv (numpy)
import numpy as np
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('Data/documentos_datafind.csv')

# Mostrar información básica del dataset
print("\nInformación del dataset:")
print(df.info())


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         10 non-null     int64 
 1   categoria  10 non-null     object
 2   titulo     10 non-null     object
 3   contenido  10 non-null     object
dtypes: int64(1), object(3)
memory usage: 452.0+ bytes
None


Genera embeddings usando TF-IDF

    ¿Qué es TF-IDF?
    - TF (Term Frequency): Cuántas veces aparece una palabra en un documento
    - IDF (Inverse Document Frequency): Qué tan rara es esa palabra en todos los documentos
    
    TF-IDF = TF × IDF
    
    Ejemplo:
    - "el", "la", "de" → Baja importancia (aparecen en todos lados)
    - "blockchain", "neurona" → Alta importancia (aparecen raramente)
    
    Esto convierte texto en números que capturan el "significado" del documento.

In [2]:
# Convertir documentos en vectores usando TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

# Configurar dimensión de los embeddings
EMBEDDING_DIM = 384  # Dimensión típica de modelos como all-MiniLM-L6-v2

def generar_embeddings_tfidf(textos, max_features=EMBEDDING_DIM):
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
    embeddings = vectorizer.fit_transform(textos).toarray()
    
    # Normalizar los vectores para que tengan norma unitaria
    # Esto es importante para calcular similitud coseno correctamente
    embeddings = normalize(embeddings, norm='l2')
    
    return embeddings, vectorizer

# Generar embeddings para los documentos
document_texts = df['contenido'].tolist()
document_embeddings, tfidf_vectorizer = generar_embeddings_tfidf(document_texts)
print(f"\nEmbeddings generados con TF-IDF. Dimensiones: {document_embeddings.shape}")




Embeddings generados con TF-IDF. Dimensiones: (10, 63)


In [6]:
# Index Builder: HSNW sin usar chromadb ycon numpy
from sklearn.neighbors import NearestNeighbors
def construir_index_hnsw(embeddings, n_neighbors=5):
    # Configurar el modelo HNSW
    hnsw_index = NearestNeighbors(
        n_neighbors=n_neighbors,
        algorithm='brute',
        metric='cosine'
    )
    
    # Ajustar el índice con los embeddings para búsqueda
    hnsw_index.fit(embeddings)
    
    return hnsw_index

# Construir el índice HNSW
hnsw_index = construir_index_hnsw(document_embeddings, n_neighbors=5)
print("\nÍndice HNSW construido con sklearn NearestNeighbors.")



Índice HNSW construido con sklearn NearestNeighbors.


In [7]:
# Query Engine: Búsqueda de documentos similares
def buscar_documentos_similares(query, index, vectorizer, top_k=5):
    # Generar embedding para la consulta
    query_embedding = vectorizer.transform([query]).toarray()
    query_embedding = normalize(query_embedding, norm='l2') # l2 es distancia euclidiana
    
    # Buscar los documentos más similares en el índice
    distances, indices = index.kneighbors(query_embedding, n_neighbors=top_k)
    
    return distances[0], indices[0]


In [10]:
# Metadata Store: Almacenar y recuperar metadatos de documentos
def obtener_metadatos_documentos(df, indices):
    metadatos = []
    for idx in indices:
        metadatos.append({
            'id': df.iloc[idx]['id'],
            'titulo': df.iloc[idx]['titulo'],
            'contenido': df.iloc[idx]['contenido']
        })
    return metadatos


In [ ]:
# Permitir que el usuario escriba una consulta y el sistema devuelva los documentos más similares.
#Y Simular embeddings de la consulta con el mismo método.
if __name__ == "__main__":
    consulta = input("\nEscribe tu consulta: ")
    
    # Buscar documentos similares
    distances, indices = buscar_documentos_similares(consulta, hnsw_index, tfidf_vectorizer, top_k=5)
    
    # Obtener metadatos de los documentos encontrados
    metadatos_encontrados = obtener_metadatos_documentos(df, indices)
    
    # Mostrar resultados
    print("\nDocumentos más similares encontrados:")
    for i, meta in enumerate(metadatos_encontrados):
        print(f"\nDocumento {i+1}:")
        print(f"ID: {meta['id']}")
        print(f"Título: {meta['titulo']}")
        print(f"Distancia (similitud coseno): {distances[i]:.4f}")
        print(f"Contenido: {meta['contenido'][:200]}...")  # Mostrar solo los primeros 200 caracteres


Documentos más similares encontrados:

Documento 1:
ID: 1
Título: Avances en IA
Distancia (similitud coseno): 0.5836
Contenido: La inteligencia artificial está transformando la industria del software....

Documento 2:
ID: 6
Título: Computación cuántica
Distancia (similitud coseno): 0.7589
Contenido: Se logran avances en la estabilidad de los qubits....

Documento 3:
ID: 5
Título: Descubrimiento espacial
Distancia (similitud coseno): 0.7644
Contenido: Un nuevo telescopio detecta exoplanetas similares a la Tierra....

Documento 4:
ID: 10
Título: Inflación
Distancia (similitud coseno): 0.7740
Contenido: La inflación global afecta los precios de productos básicos....

Documento 5:
ID: 8
Título: Gamificación
Distancia (similitud coseno): 0.7768
Contenido: Las técnicas de gamificación aumentan la motivación del estudiante....


In [19]:
# Permitir buscar solo en documentos de una categoría (VERSIÓN CORREGIDA)
def buscar_documentos_por_categoria(consulta, categoria, vectorizer, df, top_k=5):
    """
    Busca documentos similares dentro de una categoría específica
    IMPORTANTE: No recrea embeddings, usa los ya existentes
    """
    # Filtrar el DataFrame por la categoría dada
    df_filtrado = df[df['categoria'] == categoria]
    if df_filtrado.empty:
        print(f"\nNo se encontraron documentos en la categoría '{categoria}'.")
        return [], []
    
    # Obtener los índices originales de los documentos filtrados
    indices_originales = df_filtrado.index.tolist()
    
    # Usar los embeddings ya generados (NO crear nuevos)
    embeddings_filtrados = document_embeddings[indices_originales]
    
    # Construir un nuevo índice solo con los documentos de esta categoría
    n_neighbors = min(top_k, len(embeddings_filtrados))
    index_filtrado = construir_index_hnsw(embeddings_filtrados, n_neighbors=n_neighbors)
    
    # Generar embedding para la consulta usando el vectorizador ORIGINAL
    query_embedding = vectorizer.transform([consulta]).toarray()
    query_embedding = normalize(query_embedding, norm='l2')
    
    # Buscar en el índice filtrado
    distances, indices_locales = index_filtrado.kneighbors(query_embedding, n_neighbors=n_neighbors)
    
    # Mapear los índices locales a los índices originales del DataFrame
    indices_reales = [indices_originales[idx] for idx in indices_locales[0]]
    
    # Obtener metadatos usando los índices reales
    metadatos_encontrados = obtener_metadatos_documentos(df, indices_reales)
    
    return distances[0], metadatos_encontrados

In [ ]:
# Evaluar el rendimiento y calidad de la búsqueda
if __name__ == "__main__":
    consulta = input("\nEscribe tu consulta para búsqueda por categoría: ")
    categoria = input("Escribe la categoría para filtrar: ")
    
    # Buscar documentos similares en la categoría dada (SIN hnsw_index)
    distances, metadatos_encontrados = buscar_documentos_por_categoria(
        consulta, 
        categoria, 
        tfidf_vectorizer,
        df, 
        top_k=5
    )
    
    # Mostrar resultados
    if metadatos_encontrados:
        print(f"\nDocumentos más similares encontrados en la categoría '{categoria}':")
        for i, meta in enumerate(metadatos_encontrados):
            print(f"\n{'='*60}")
            print(f"Documento {i+1}:")
            print(f"ID: {meta['id']}")
            print(f"Título: {meta['titulo']}")
            print(f"Distancia (similitud coseno): {distances[i]:.4f}")
            print(f"Contenido: {meta['contenido'][:200]}...")
            if distances[i] < 0.5:  # Umbral de similitud
                print("Este documento es relevante para tu consulta")
            else:
                print("Este documento es relevante, pero podría no ser el más adecuado.")
    else:
        print(f"\nNo se encontraron documentos en la categoría '{categoria}'.")


Documentos más similares encontrados en la categoría 'Tecnología':

Documento 1:
ID: 6
Título: Computación cuántica
Distancia (similitud coseno): 0.5278
Contenido: Se logran avances en la estabilidad de los qubits....

Documento 2:
ID: 1
Título: Avances en IA
Distancia (similitud coseno): 0.7874
Contenido: La inteligencia artificial está transformando la industria del software....
